In [4]:
example_survey_response = "The recertification process was great, but to receive our letter of approval took an extended amount of time."


In [5]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
# simple sentiment

sentiment_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/stanford-sentiment-treebank-roberta.2021-03-11.tar.gz")
sentiment_predictor.predict(example_survey_response)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'logits': [-2.9592537879943848, 2.9868783950805664],
 'probs': [0.002609113696962595, 0.9973909258842468],
 'token_ids': [0,
  133,
  3872,
  2399,
  5000,
  609,
  21,
  372,
  6,
  53,
  7,
  1325,
  84,
  1601,
  9,
  2846,
  362,
  41,
  3112,
  1280,
  9,
  86,
  4,
  2],
 'label': '0',
 'tokens': ['<s>',
  'The',
  'Ġrec',
  'ert',
  'ification',
  'Ġprocess',
  'Ġwas',
  'Ġgreat',
  ',',
  'Ġbut',
  'Ġto',
  'Ġreceive',
  'Ġour',
  'Ġletter',
  'Ġof',
  'Ġapproval',
  'Ġtook',
  'Ġan',
  'Ġextended',
  'Ġamount',
  'Ġof',
  'Ġtime',
  '.',
  '</s>']}

In [7]:
# example dataframe with text
df = pd.read_csv("https://raw.githubusercontent.com/GSA/GovCXAnalyzer/main/data/example-video-game-reviews.csv")
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,4.0,True,"08 25, 2014",A3UIDVXLPX9JRN,B000035XGF,John Hiebs,Tough old game but very fun annnd in great sha...,Four Stars for Castlevania Classicness!,1408924800,NaN,NaN,NaN
1,5.0,True,"12 25, 2014",AGS25M1ED8EGZ,B00K848IH0,sally85,love it,great,1419465600,NaN,"{'Edition:': ' Alpha Sapphire', 'Platform:': '...",NaN
2,5.0,True,"12 18, 2015",A10K49BDURK2Z,B00ZJRHSZO,Doug Johnston,Better than any other remote made for PS4!,BEST OUT THERE!,1450396800,NaN,NaN,NaN
3,5.0,True,"07 16, 2015",A1WYHM3323PC7E,B00IQCRKW0,calob snow,game is friggin awsome!,Five Stars,1437004800,NaN,{'Format:': ' Video Game'},NaN
4,2.0,False,"05 24, 2009",ALTET8PNJHSSG,B001MBUGLY,PRT,"The software as good as it is, can only go so ...",Frustrated,1243123200,14.0,{'Edition:': ' Standard'},NaN


In [27]:
df.loc[4]["reviewText"]

'The software as good as it is, can only go so far.  You would think EA would take more time to test the leg strap, and realize how frustrating for everyone heavy or thin, for this thing to stay on.  I\'ve tried all the suggestions, but the strap still falls off.  I think I\'m getting more of a work out, picking up the strap, and trying endless ways to keep the strap on, then actually working out.  Seriously!  Finally, I stopped using the strap and figuring out ways to work with only the wii motes and nun chuck...actually way more liberating!  I wanted to love this game, but come on EA think of the consumer!  There are quite a few "honest" reviews here who don\'t overlook this.  If you are willing to spend most of your workout picking up your leg strap, then I guess this won\'t disappoint.  As for me, I expected more...especially if the leg strap is a vital part of each work out!'

In [30]:
# for list of sentiment predictors
l = []
# put your list of texts

list_of_texts = [example_survey_response,
                 "Tough old game but very fun annnd in great shape... Quatro stars. Woulda' been five but the level of frustration to beat this game brings it down to a four lol."]



for s in list_of_texts:
    
    preds = sentiment_predictor.predict(s)
    preds['orig_input'] = s
    l.append(preds)

In [31]:
# creating a new dataframe with positive vs negative simple sentiment
simplesentiment = pd.DataFrame(l)

simplesentiment["simple_sentiment_pos_prob"] = simplesentiment["probs"].map(lambda x: x[0])
simplesentiment["simple_sentiment_neg_prob"] = simplesentiment["probs"].map(lambda x: x[1])


In [32]:
# now using a question answering model
predictorqa = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/transformer-qa.2021-02-11.tar.gz")

def make_predqa(passage, question, predictorqa):
    return predictorqa.predict(
    passage=passage,#"The Matrix is a 1999 science fiction action film written and directed by The Wachowskis, starring Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, and Joe Pantoliano.",
    question=question #"Who stars in The Matrix?"
)

In [33]:
newcol = []

# asking it to parse out the spans of text of what was frustrating
for i,row in simplesentiment.iterrows():
    passage = row['orig_input']
    question = 'What was frustrating?'
    newd = {}
    output_ = make_predqa(passage, question, predictorqa)
    newd['model_input'] = {"passage": passage, "question": question}
    newd['model_output'] = output_
    newcol.append(newd)
simplesentiment[f"MODELQA: {question}"] = newcol


newcol = []
# asking it to parse out what is pleasant 
# feel free to change around questions for different types of extractions
for i,row in simplesentiment.iterrows():
    passage = row['orig_input']
    question = 'What was pleasant?'
    newd = {}
    output_ = make_predqa(passage, question, predictorqa)
    newd['model_input'] = {"passage": passage, "question": question}
    newd['model_output'] = output_
    newcol.append(newd)
simplesentiment[f"MODELQA: {question}"] = newcol
simplesentiment

,logits,probs,token_ids,label,tokens,orig_input,simple_sentiment_pos_prob,simple_sentiment_neg_prob,MODELQA: What was frustrating?,MODELQA: What was pleasant?
0,"[-2.9592537879943848, 2.9868783950805664]","[0.002609113696962595, 0.9973909258842468]","[0, 133, 3872, 2399, 5000, 609, 21, 372, 6, 53...",0,"[<s>, The, Ġrec, ert, ification, Ġprocess, Ġwa...","The recertification process was great, but to ...",0.002609,0.997391,{'model_input': {'passage': 'The recertificati...,{'model_input': {'passage': 'The recertificati...
1,"[0.9527846574783325, -1.4331105947494507]","[0.9157454371452332, 0.08425460010766983]","[0, 565, 4894, 793, 177, 53, 182, 1531, 9915, ...",1,"[<s>, T, ough, Ġold, Ġgame, Ġbut, Ġvery, Ġfun,...",Tough old game but very fun annnd in great sha...,0.915745,0.084255,{'model_input': {'passage': 'Tough old game bu...,{'model_input': {'passage': 'Tough old game bu...


In [34]:
simplesentiment['MODELQA: What was pleasant? STRING'] = (
    simplesentiment['MODELQA: What was pleasant?'].map(lambda x: x['model_output']['best_span_str'])
)
simplesentiment['MODELQA: What was frustrating? STRING'] = (
    simplesentiment['MODELQA: What was frustrating?'].map(lambda x: x['model_output']['best_span_str'])
)

In [35]:
simplesentiment

,logits,probs,token_ids,label,tokens,orig_input,simple_sentiment_pos_prob,simple_sentiment_neg_prob,MODELQA: What was frustrating?,MODELQA: What was pleasant?,MODELQA: What was pleasant? STRING,MODELQA: What was frustrating? STRING
0,"[-2.9592537879943848, 2.9868783950805664]","[0.002609113696962595, 0.9973909258842468]","[0, 133, 3872, 2399, 5000, 609, 21, 372, 6, 53...",0,"[<s>, The, Ġrec, ert, ification, Ġprocess, Ġwa...","The recertification process was great, but to ...",0.002609,0.997391,{'model_input': {'passage': 'The recertificati...,{'model_input': {'passage': 'The recertificati...,The recertification process,to receive our letter of approval took an exte...
1,"[0.9527846574783325, -1.4331105947494507]","[0.9157454371452332, 0.08425460010766983]","[0, 565, 4894, 793, 177, 53, 182, 1531, 9915, ...",1,"[<s>, T, ough, Ġold, Ġgame, Ġbut, Ġvery, Ġfun,...",Tough old game but very fun annnd in great sha...,0.915745,0.084255,{'model_input': {'passage': 'Tough old game bu...,{'model_input': {'passage': 'Tough old game bu...,,to beat this game


In [36]:
from textblob import TextBlob


In [40]:
blob = TextBlob("the recertification process")
blob.noun_phrases

WordList(['recertification process'])

In [46]:
import spacy

nlp = spacy.load("en_core_web_md")

In [51]:
list(nlp("to beat this game").noun_chunks)

[this game]

In [53]:
simplesentiment["pleasant_noun_chunks"] = simplesentiment["MODELQA: What was pleasant? STRING"].map(lambda x: list(nlp(str(x)).noun_chunks))
simplesentiment["frustrating_noun_chunks"] = simplesentiment["MODELQA: What was frustrating? STRING"].map(lambda x: list(nlp(str(x)).noun_chunks))

                    

In [54]:
# noun phrases are not 
simplesentiment


,logits,probs,token_ids,label,tokens,orig_input,simple_sentiment_pos_prob,simple_sentiment_neg_prob,MODELQA: What was frustrating?,MODELQA: What was pleasant?,MODELQA: What was pleasant? STRING,MODELQA: What was frustrating? STRING,pleasant_noun_chunks,frustrating_noun_chunks
0,"[-2.9592537879943848, 2.9868783950805664]","[0.002609113696962595, 0.9973909258842468]","[0, 133, 3872, 2399, 5000, 609, 21, 372, 6, 53...",0,"[<s>, The, Ġrec, ert, ification, Ġprocess, Ġwa...","The recertification process was great, but to ...",0.002609,0.997391,{'model_input': {'passage': 'The recertificati...,{'model_input': {'passage': 'The recertificati...,The recertification process,to receive our letter of approval took an exte...,"[(The, recertification, process)]","[(our, letter), (approval), (an, extended, amo..."
1,"[0.9527846574783325, -1.4331105947494507]","[0.9157454371452332, 0.08425460010766983]","[0, 565, 4894, 793, 177, 53, 182, 1531, 9915, ...",1,"[<s>, T, ough, Ġold, Ġgame, Ġbut, Ġvery, Ġfun,...",Tough old game but very fun annnd in great sha...,0.915745,0.084255,{'model_input': {'passage': 'Tough old game bu...,{'model_input': {'passage': 'Tough old game bu...,,to beat this game,[],"[(this, game)]"
